In [26]:
import pickle as pkl
from diskchef import UVFits
from pathlib import Path
import astropy.units as u


IMAGER_EXEC = "bash -lc imager"

### Create uvfits file from the model output fits file using the information in the observation uvfits files


In [33]:
# data_to_write = Path('Reference/CITau_100_-2_40_30_0.55/radmc_gas/CO J=2-1_image.fits')
disk = "DMTau"
for mol in ["CO", "13CO", "C18O"]:
    script = str("""
                            FITS "{input_file}" TO "{name}"
                            READ UV "{name}"
                            UV_MAP
                            CLEAN
                            LUT {lut}
                            ! VIEW CLEAN /NOPAUSE
                            LET SIZE 10
                            LET DO_CONTOUR NO
                            SHOW CLEAN
                            HARDCOPY "{name}.{device}" /DEVICE {device} /OVERWRITE
                            WRITE CLEAN imaged
                            FITS "{name}.fits" from imaged.lmv-clean /OVERWRITE

                    """)
# DLTau_600_-1.3_38_10_0.55
#     for res in ['', '_residuals']:
    for res in ['']:
        data_to_write = Path(
            f'/Users/franceschi/mysims/diskchef_fitting_greg/Reference/TEST_DMTau_200_-1.4_31_21_0.55/radmc_gas/{mol} '
            f'J=2-1_image'
            f'.fits'
        )
        file_to_modify = Path(f'observations_new/{disk}/{mol}_cut.uvfits')
        output = data_to_write.parents[1]  / f'{file_to_modify.stem}_visibilities{res}.uvfits'
        _res = False
        if res == '_residuals':
            _res = True
        UVFits.write_visibilities_to_uvfits(data_to_write, file_to_modify, output, residual=_res)
        UVFits.run_imaging(output, f'{mol}_cut{res}', imager_executable=IMAGER_EXEC, script_template=script, device='png')
        print(output)

/Users/franceschi/mysims/diskchef_fitting_greg/Reference/TEST_DMTau_200_-1.4_31_21_0.55/CO_cut_visibilities.uvfits
/Users/franceschi/mysims/diskchef_fitting_greg/Reference/TEST_DMTau_200_-1.4_31_21_0.55/13CO_cut_visibilities.uvfits
/Users/franceschi/mysims/diskchef_fitting_greg/Reference/TEST_DMTau_200_-1.4_31_21_0.55/C18O_cut_visibilities.uvfits


First write the visibilities from the observations to the model fits file `write_visibilities_to_uvfits`
Then it convolves it like if it was observed

In [188]:
data_to_write = Path('Reference/CITau_200_-2.6_60_7_0.55')
# data_to_write = Path('fit/test/radmc_gas/13CO J=2-1_image.fits')
file_to_modify = Path('observations_new/CITau/CO_cut.uvfits')

model_visibilities = data_to_write.parents[1]  / f'{data_to_write.stem}_visibilities.uvfits'
UVFits.write_visibilities_to_uvfits(data_to_write, file_to_modify, model_visibilities)

script = str("""
                        FITS "{input_file}" TO "{name}"
                        READ UV "{name}"
                        UV_MAP
                        CLEAN
                        LUT {lut}
                        ! VIEW CLEAN /NOPAUSE
                        LET SIZE 10
                        LET DO_CONTOUR NO
                        SHOW CLEAN
                        HARDCOPY "{name}.{device}" /DEVICE {device} /OVERWRITE
                        WRITE CLEAN imaged
                        FITS "{name}.fits" from imaged.lmv-clean /OVERWRITE

                """)


UVFits.run_imaging(model_visibilities, 'CO_cut', imager_executable=IMAGER_EXEC, script_template=script, device='png')

FileNotFoundError: [Errno 2] No such file or directory: 'Reference/CITau_200_-2.6_60_7_0.55'

Read and clean uvt, save uvfits

In [ ]:
model_visibilities = 'observations_test/DNTau/s-Line-22-CO_1+D_cut.uvt'
output = 'observations_test/DNTau/s-Line-22-CO_1+D_cut.uvt'

script = str("""
                        READ UV "{input_file}"
                        UV_MAP
                        CLEAN
                        LUT {lut}
                        ! VIEW CLEAN /NOPAUSE
                        LET SIZE 10
                        LET DO_CONTOUR NO
                        SHOW CLEAN
                        HARDCOPY "{name}.{device}" /DEVICE {device} /OVERWRITE
                        WRITE UV "{name}"
                        FITS "{name}.uvfits" FROM "{name}.uvt" /OVERWRITE /STYLE CASA

                """)

UVFits.run_imaging(model_visibilities, output, imager_executable=IMAGER_EXEC, script_template=script, device='png',)